In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rohan\\Desktop\\Machine Learning Projects\\Test summirization\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rohan\\Desktop\\Machine Learning Projects\\Test summirization\\Text-Summarization'

In [5]:
# %pip install -r requirements.txt

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directory


In [8]:
class configurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directory([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation

        create_directory([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config






In [13]:
# %pip uninstall transformers
# %pip install transformers
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 547.8/547.8 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 378.7/378.7 kB 5.9 MB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 316.1/316.1 kB 6.5 MB/s eta 0:00:00
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
     ---------------------------------------- 50.7/50.7 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 76.9/76.9 kB ? eta 0:00:00
     -------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [14]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


[2024-08-09 13:40:27,318: INFO: utils: NumExpr defaulting to 8 threads.]
[2024-08-09 13:40:28,117: INFO: config: PyTorch version 2.3.0 available.]
[2024-08-09 13:40:28,119: INFO: config: Polars version 0.20.25 available.]
[2024-08-09 13:40:28,128: INFO: config: TensorFlow version 2.16.1 available.]


In [16]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        

In [17]:
try:
    config = configurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-08-09 13:59:36,612: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-09 13:59:36,637: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-09 13:59:36,643: INFO: common: created directory at: artifacts]
[2024-08-09 13:59:36,646: INFO: common: created directory at: artifacts/data_transformation]


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

C:\Users\rohan\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rohan\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

...

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]